In [1]:
import pandas as pd
df=pd.read_csv("../data/sierraleone-bumbuna.csv")
df.describe()

,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,201.957515,116.376337,113.720571,206.643095,198.114691,26.319394,79.448857,1.146113,1.691606,0.363823,133.044668,7.172220,999.876469,0.000967,0.004806,32.504263,32.593091,NaN
std,298.495150,218.652659,158.946032,300.896893,288.889073,4.398605,20.520775,1.239248,1.617053,0.295000,114.284792,7.535093,2.104419,0.031074,0.047556,12.434899,12.009161,NaN
min,-19.500000,-7.800000,-17.900000,0.000000,0.000000,12.300000,9.900000,0.000000,0.000000,0.000000,0.000000,0.000000,993.000000,0.000000,0.000000,10.700000,11.100000,NaN
25%,-2.800000,-0.300000,-3.800000,0.000000,0.000000,23.100000,68.700000,0.000000,0.000000,0.000000,0.000000,0.000000,999.000000,0.000000,0.000000,23.500000,23.800000,NaN
50%,0.300000,-0.100000,-0.100000,3.600000,3.400000,25.300000,85.400000,0.800000,1.600000,0.400000,161.500000,6.200000,1000.000000,0.000000,0.000000,26.600000,26.900000,NaN
75%,362.400000,107.000000,224.700000,359.500000,345.400000,29.400000,96.700000,2.000000,2.600000,0.600000,234.100000,12.000000,1001.000000,0.000000,0.000000,40.900000,41.300000,NaN
max,1499.000000,946.000000,892.000000,1507.000000,1473.000000,39.900000,100.000000,19.200000,23.900000,4.100000,360.000000,98.400000,1006.000000,1.000000,2.400000,72.800000,70.400000,NaN


In [2]:
missing_values=df.isna().sum()
percent_missing=(missing_values/len(df))*100
missing_report = pd.DataFrame({
    'Missing Values': missing_values,
    '% of Total': percent_missing
})
missing_report[missing_report['% of Total'] > 5]

,Missing Values,% of Total
Comments,525600,100.0


In [3]:
from scipy.stats import zscore

cols_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
z_scores = df[cols_to_check].apply(zscore)

# Identify rows with Z-score > 3 or < -3
outliers = (z_scores.abs() > 3)
df_outliers = df[outliers.any(axis=1)]
print(df_outliers)

               Timestamp    GHI    DNI    DHI   ModA   ModB  Tamb    RH   WS  \
653     2021-10-30 10:54  814.0  257.0  594.3  878.0  856.0  25.9  85.3  0.5   
654     2021-10-30 10:55  826.0  263.5  599.6  883.0  861.0  26.0  85.4  1.7   
655     2021-10-30 10:56  851.0  285.0  605.9  912.0  890.0  26.0  84.2  1.6   
656     2021-10-30 10:57  823.0  254.8  603.9  878.0  856.0  26.0  85.3  0.8   
657     2021-10-30 10:58  810.0  244.5  598.8  864.0  843.0  26.1  85.7  0.6   
...                  ...    ...    ...    ...    ...    ...   ...   ...  ...   
522184  2022-10-27 15:05  167.2   -0.5  165.1  167.6  160.9  30.4  68.3  5.3   
522185  2022-10-27 15:06  163.4   -0.6  161.0  165.7  159.2  30.2  67.7  7.1   
522194  2022-10-27 15:15  183.5    0.3  181.6  173.7  168.7  29.3  70.2  4.9   
523792  2022-10-28 17:53   31.3   -0.5   30.2   34.3   32.3  28.4  79.5  5.0   
523794  2022-10-28 17:55   22.8   -0.8   21.4   26.5   25.0  27.9  80.4  5.0   

        WSgust  WSstdev     WD  WDstdev

In [4]:
# Impute missing values in key columns with median
for col in cols_to_check:
 df[col] = df[col].fillna(df[col].median())


# Drop rows if Timestamp or target values are missing
df.dropna(subset=['Timestamp', 'GHI', 'DNI', 'DHI'], inplace=True)

In [5]:
df.to_csv("../data/sierraleone-bumbuna_clean.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure Timestamp is datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Set style
sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# Plot GHI, DNI, DHI, Tamb over time (use a sample for performance)
sample_df = df.sample(5000).sort_values('Timestamp')

plt.figure(figsize=(16, 8))
plt.plot(sample_df['Timestamp'], sample_df['GHI'], label='GHI', alpha=0.6)
plt.plot(sample_df['Timestamp'], sample_df['DNI'], label='DNI', alpha=0.6)
plt.plot(sample_df['Timestamp'], sample_df['DHI'], label='DHI', alpha=0.6)
plt.plot(sample_df['Timestamp'], sample_df['Tamb'], label='Tamb', alpha=0.6)
plt.legend()
plt.title('Solar Irradiance & Temperature Over Time')
plt.xlabel('Time')
plt.ylabel('W/m² or °C')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Group by Cleaning status and compute average ModA and ModB
cleaning_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean().reset_index()

# Bar plot of average ModA and ModB by Cleaning status
plt.figure(figsize=(10, 5))
cleaning_impact.plot(x='Cleaning', kind='bar', legend=True)
plt.title('Impact of Cleaning on ModA and ModB Readings')
plt.xlabel('Cleaning Status (0 = No, 1 = Yes)')
plt.ylabel('Average Sensor Reading (W/m²)')
plt.xticks(rotation=0)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
correlation_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
correlation_matrix = df[correlation_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of Solar and Module Temperatures")
plt.show()

In [ ]:
# Scatter plots to examine environmental factors vs GHI
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
sns.scatterplot(data=df, x='WS', y='GHI', ax=axs[0, 0])
axs[0, 0].set_title('Wind Speed vs. GHI')

sns.scatterplot(data=df, x='WSgust', y='GHI', ax=axs[0, 1])
axs[0, 1].set_title('Wind Gust vs. GHI')

sns.scatterplot(data=df, x='WD', y='GHI', ax=axs[1, 0])
axs[1, 0].set_title('Wind Direction vs. GHI')

sns.scatterplot(data=df, x='RH', y='Tamb', ax=axs[1, 1])
axs[1, 1].set_title('Relative Humidity vs. Ambient Temp')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from windrose import WindroseAxes

# Clean data: drop NaNs
wind_data = df.dropna(subset=['WS', 'WD'])

ax = WindroseAxes.from_ax()
ax.bar(wind_data['WD'], wind_data['WS'], normed=True, opening=0.8, edgecolor='white', cmap=plt.cm.viridis)
ax.set_legend()
plt.title("Wind Rose: Speed vs. Direction")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(df['GHI'], bins=30, kde=True, ax=axs[0], color="skyblue")
axs[0].set_title("Distribution of GHI")

sns.histplot(df['WS'], bins=30, kde=True, ax=axs[1], color="orange")
axs[1].set_title("Distribution of Wind Speed")

plt.tight_layout()
plt.show()

In [ ]:
df_daylight = df[df['GHI'] > 0].copy()

In [ ]:
# RH vs Tamb
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_daylight, x='RH', y='Tamb', alpha=0.5)
plt.title('RH vs. Ambient Temperature')
plt.xlabel('Relative Humidity (%)')
plt.ylabel('Ambient Temperature (°C)')
plt.tight_layout()
plt.show()

# Bubble chart: GHI vs Tamb, bubble size = RH
plt.figure(figsize=(10, 6))
plt.scatter(df_daylight['GHI'], df_daylight['Tamb'], s=df_daylight['RH'], alpha=0.3, c='green')
plt.title('GHI vs Tamb with RH as Bubble Size')
plt.xlabel('Global Horizontal Irradiance (GHI)')
plt.ylabel('Ambient Temperature (°C)')
plt.tight_layout()
plt.show()